In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# from sklearn import ?
# from sklearn.metrics import ?

<br>

## 1. Preparing dataset (2번부터 실습 진행)

In [3]:
data_df = pd.read_csv('titanic.csv')
data_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## Data info

- **PassengerId** : Unique ID of passenger
- **Survived** : 0 = No, 1 = Yes
- **pclass** : Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd)
- **sibsp** : # of siblings & spouses aboard the Titanic
- **parch** : # of parents / children aboard the Titanic
- **ticket** : Ticket number
- **cabin** : Cabin number
- **embarked** : Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

In [4]:
y_data = data_df[['Survived']]
# y_data.head(3)

In [5]:
del data_df['Survived']
x_data = data_df.copy()
# x_data.head(3)

<br>

## 2. Feature engineering & Feature selection

#### 시도해볼 수 있는 전략들

- 불필요한 열이나 예측에 방해가 되는 열은 아예 지우기 (ex. PassengerId)
- 결측치 채우기 
- Text로 되어있는 Category(Factor)는 숫자로 바꿔주기 (ex. Male/Female -> 0/1)
- 필요한 경우 기존 열을 바탕으로 새로운 열을 계산해 추가하기


- (실수 범위를 구간 범위로 바꿔주기 -> 데이터 손실 발생 가능)

> 결측치 처리: ex. 보험사에서 다루는 고객 데이터셋 중 나이열('age')에 있는 결측치를 다룰 때

    - 랜덤 값으로 채우기
    - 대표 값으로 채우기 (mean, median, mode)
    - 위아래 데이터의 평균값으로 채우기
    - 행들을 그룹으로 묶은 후, 해당 그룹의 대표값으로 채우기
    - 머신러닝 예측모델을 만들어 해당 열의 결측치를 채우기
    - 특정 기준 이상의 결측치가 있을 시 해당 열 삭제

In [6]:
x_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [7]:
# # 기본적인 결측치 채우는 방법
# mean_age = x_data['Age'].median(skipna=True)  #  Null == None == N/A(not available) == NaN(Not a Number) == 無(vacant)
# x_data['Age'] = x_data['Age'].fillna(mean_age) # or .fillna(int/float/str 값)

In [8]:
# 결측치 확인
x_data.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

======================

1. cabin O/X
2. age o/x
==> 4가지 경우의 수로 비교해보자.
3.  Embarked 2개니까 그냥 행 삭제 ( pclass 확인하고 나서 1등급이기보단 3등급일 확률이 높으니 대체해보자.)

1. Embarked 2개 survived랑 관련이 있는 지 그래프로 확인
2. 관련이 없는 것으로 판정이 난다면, 2개 행 삭제
3. 4가지 경우의 수로 비교 (cabin O/X & age o/x)
4. name -> ms, mr, dr 등 신분 추출

======================

### (1) EDA 및 시각화

In [11]:
import matplotlib.pyplot as plt 
import seaborn as sns

%matplotlib inline # plt.show()를 하지 않아도 자동으로 생성되도록 만든다.

In [14]:
# y_data('survived') + x_data 모두 있는 df 다시 불러오기
data = pd.read_csv('titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


https://wikidocs.net/187709

In [17]:
# plotly 라이브러리 설치
!pip install plotly

# plotly 라이브러리 호출
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

# 기본 폰트 설정
layout_font = {'font':dict(size=18,color='#60606e',family='Franklin Gothic' )}

In [156]:
# 1. 성별 생존 비율
fig = px.pie(data, values='Survived', names='Sex', 
             title='<b>성별 생존 비율<b>',
             color_discrete_sequence=px.colors.sequential.YlOrRd)

# 도넛 모양 pie 차트
fig.update_traces(hole=.3)

# 조각 내기
fig.update_traces(pull=[0.1, 0])

# 내부 텍스트 편집
fig.update_traces(textposition='outside',textinfo='label+percent',
                  textfont_size=15,
                  textfont_color="black")


fig.show()

In [120]:
# 2. 좌석등급별 생존 비율
fig = px.pie(data, values='Survived', names='Pclass', 
             title='<b>좌석등급별 생존 비율<b>',
             color_discrete_sequence=px.colors.sequential.Reds)

# 도넛 모양 pie 차트
fig.update_traces(hole=.3)

# 조각 내기
fig.update_traces(pull=[0, 0.1, 0])

# 내부 텍스트 편집
fig.update_traces(textposition='outside',textinfo='label+percent',
                  textfont_size=15,
                  textfont_color="black")


fig.show()  # 3등급 승객 생존율이 2등급 승객 생존율보다 높은 이유는?

In [105]:
# 좌석등급별 생존자 성별 분포

male_df = data[data['Sex'] == 'male']
female_df = data[data['Sex'] == 'female']

fig = px.bar(male_df, x='Sex', color='Pclass',
             title='좌석등급별 남성 생존율')


fig.show()  # 3등급 승객 생존율이 2등급 승객 생존율보다 높은 이유는?

In [121]:
# 3. 형제자매 수별 생존 비율

fig = px.pie(data, values='Survived', names='SibSp', 
             title='<b>형제자매 수별 생존 비율<b>',
             color_discrete_sequence=px.colors.sequential.Blues)

# 도넛 모양 pie 차트
fig.update_traces(hole=.3)


# 내부 텍스트 편집
fig.update_traces(textposition='inside',textinfo='percent',
                  textfont_size=10,
                  textfont_color="black")


fig.show() 

In [155]:
# 4. 연령별 생존 비율

# (1) Age 결측치를 0으로 대체한 후,
survived_df = data[data.Survived == 1]
survived_df = survived_df[['Pclass', 'Sex', 'Age']]
# survived_df.isna().sum()
survived_df = survived_df.fillna({'Age':0})
survived_df = survived_df.astype({'Age':'int64'})
# survived_df.isnull().sum()

survived_df

# (2) 연령별 생존비율 시각화
# 
# fig = px.line(survived_df, x='Age', y='Sex')
# fig.show()


# (3) 좌석등급별 생존비율 시각화


# (4) 연령-좌석등급별 생존비율 시각화

,Pclass,Sex,Age
1,1,female,38
2,3,female,26
3,1,female,35
8,3,female,27
9,2,female,14
...,...,...,...
875,3,female,15
879,1,female,56
880,2,female,25
887,1,female,19


In [158]:
# 좌석등급별 성별 분포
fig = px.bar(data, x='Pclass',
             hover_data=['Sex'], 
             color='Sex')

fig = go.Figure(data=[go.Bar(name='')]
               )

fig.update_layout(title='<b>좌석등급별 성별 분포</b>')
fig.show()


<br>

## 2. Train - Test split (비율을 7:3 으로 유지해주시고, seed는 0을 적용해주세요)

<br>

## 3. Create model instance variable (동시에 여러 모델을 다른 이름으로 만들 수 있습니다.)

<br>

## 4. Train the model

<br>

## 5. Predict on test data & Check the result with metrics (모델 간 비교가 가능합니다.)